In [1]:
import os
import sys
sys.path.append('..')
sys.path.append('../../Automatic-Circuit-Discovery/')
import IPython
ipython = get_ipython()
if ipython is not None:
    ipython.magic('load_ext autoreload')
    ipython.magic('autoreload 2')
import torch as t
from torch import Tensor

from acdc.hybridretrieval.utils import (
    get_all_hybrid_retrieval_things,
    get_gpt2_small
)

from transformer_lens import HookedTransformer, ActivationCache

import tqdm.notebook as tqdm
import plotly

device = t.device("cuda" if t.cuda.is_available() else "CPU")
print(device)

/tmp/ipykernel_257450/3208339110.py:8: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic('load_ext autoreload')
/tmp/ipykernel_257450/3208339110.py:9: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic('autoreload 2')
/home/iustin/.conda/envs/eap/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [2]:
all_kbicr_items = get_all_hybrid_retrieval_things(num_examples=20, device=device, metric_name='logit_diff')

tl_model = all_kbicr_items.tl_model
validation_metric = all_kbicr_items.validation_metric
validation_data = all_kbicr_items.validation_data
validation_labels = all_kbicr_items.validation_labels
validation_patch_data = all_kbicr_items.validation_patch_data
test_metrics = all_kbicr_items.test_metrics
test_data = all_kbicr_items.test_data
test_labels = all_kbicr_items.test_labels
test_patch_data = all_kbicr_items.test_patch_data

Loaded pretrained model gpt2 into HookedTransformer
Moving model to device:  cuda
Clean Prompts:
Alice lives in France, Paris - Alice, John lives in Germany, Berlin - John, Peter lives in USA, Washington - Peter
Lucy lives in Turkey, Ankara - Lucy, Sara lives in Italy, Rome - Sara, Bob lives in Spain, Madrid - Bob
Tom lives in Canada, Toronto - Tom, Anna lives in Australia, Canberra - Anna, Michael lives in Japan, Tokyo - Michael
David lives in Brazil, Rio de Janeiro - David, Alice lives in France, Paris - Alice, Peter lives in Germany, Berlin - Peter
Sara lives in USA, Washington - Sara, Lucy lives in Turkey, Ankara - Lucy, Tom lives in Italy, Rome - Tom
John lives in Spain, Madrid - John, Michael lives in Canada, Toronto - Michael, Anna lives in Australia, Canberra - Anna
David lives in Japan, Tokyo - David, Sara lives in Brazil, Rio de Janeiro - Sara, Alice lives in France, Paris - Alice
Bob lives in Germany, Berlin - Bob, Peter lives in USA, Washington - Peter, Tom lives in Turkey,

In [3]:
model = HookedTransformer.from_pretrained(
    'gpt2-small',
    center_writing_weights=False,
    center_unembed=False,
    fold_ln=False,
    device=device,
)
model.set_use_hook_mlp_in(True)
model.set_use_split_qkv_input(True)
model.set_use_attn_result(True)

Loaded pretrained model gpt2-small into HookedTransformer


In [4]:
def abs_kbicr_metric(logits):
    return -abs(test_metrics['logit_diff'](logits))

In [5]:
os.getcwd()

'/home/iustin/Mech-Interp/Automatic-Circuit-Discovery'

In [6]:
os.chdir('../edge-attribution-patching')
os.getcwd()

'/home/iustin/Mech-Interp/edge-attribution-patching'

In [7]:
from ACDCPPExperiment import ACDCPPExperiment
import numpy as np
THRESHOLDS = [0.09] # np.arange(0.04, 0.16, 0.005)
# I'm just using one threshold so I can move fast!

tl_model.reset_hooks()
RUN_NAME = 'abs_edges'
acdcpp_exp = ACDCPPExperiment(
    tl_model,
    test_data,
    test_patch_data,
    test_metrics['logit_diff'],
    abs_kbicr_metric,
    THRESHOLDS,
    # run_name=RUN_NAME,
    verbose=False,
    attr_absolute_val=True,
    save_graphs_after=0,
    pruning_mode='edge',
    no_pruned_nodes_attr=1,
)
pruned_heads, num_passes, pruned_attrs = acdcpp_exp.run()

  0%|          | 0/1 [00:00<?, ?it/s]

ln_final.hook_normalized
ln_final.hook_scale
blocks.11.hook_resid_post
blocks.11.hook_mlp_out
blocks.11.mlp.hook_post
blocks.11.mlp.hook_pre
blocks.11.ln2.hook_normalized
blocks.11.ln2.hook_scale
blocks.11.hook_mlp_in
blocks.11.hook_resid_mid
blocks.11.hook_attn_out
blocks.11.attn.hook_result
blocks.11.attn.hook_z
blocks.11.attn.hook_pattern
blocks.11.attn.hook_attn_scores
blocks.11.attn.hook_v
blocks.11.attn.hook_k
blocks.11.attn.hook_q
blocks.11.ln1.hook_normalized
blocks.11.ln1.hook_scale
blocks.11.hook_v_input
blocks.11.hook_k_input
blocks.11.hook_q_input
blocks.11.hook_resid_pre
blocks.10.hook_resid_post
blocks.10.hook_mlp_out
blocks.10.mlp.hook_post
blocks.10.mlp.hook_pre
blocks.10.ln2.hook_normalized
blocks.10.ln2.hook_scale
blocks.10.hook_mlp_in
blocks.10.hook_resid_mid
blocks.10.hook_attn_out
blocks.10.attn.hook_result
blocks.10.attn.hook_z
blocks.10.attn.hook_pattern
blocks.10.attn.hook_attn_scores
blocks.10.attn.hook_v
blocks.10.attn.hook_k
blocks.10.attn.hook_q
blocks.10.ln

  0%|          | 0/1 [00:02<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 12.00 MiB. GPU 0 has a total capacty of 7.79 GiB of which 7.38 MiB is free. Including non-PyTorch memory, this process has 7.77 GiB memory in use. Of the allocated memory 6.81 GiB is allocated by PyTorch, and 791.11 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

: 